In [11]:
!pip install graphlearning
!pip install annoy
import numpy as np
from scipy.optimize import minimize, LinearConstraint
import sklearn.datasets as datasets
import graphlearning as gl
import matplotlib.pyplot as plt
import time, pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
"""
Returns the ith vector of the usual basis of R^k
Index starts at 0
Note: When idx is a (n, ) array (of indices), then it returns
an (n, k) array whose j-th row is the idx[j]-th vector of the
basis of R^k

### Example
# print(euclidean_basis(0, 5))
# print(euclidean_basis(2, 5))
# print(euclidean_basis([0, 2], 5))
"""

def save_models(models, filename = "mnist_models.pickle"):
    with open(filename, 'wb') as f:
        pickle.dump(models, f)
        f.close()
    
    return

def load_models(filename = "mnist_models.pickle"):
    with open(filename, "rb") as f:
        models = pickle.load(f)
        f.close()
    
    return models

def euclidean_basis(idx, k):
    eye = np.eye(k)
    return eye[idx]

def vectorized_jacobian(u_flattened, W, idx, y, p):
    n = W.shape[0]
    k = u_flattened.size//n
    m = y.shape[0]    
    
    u = u_flattened.reshape((n,k))
    gradu = graph_grad(u)
    normed_gradu = np.apply_along_axis(np.linalg.norm, 2, gradu)
    y_bar = y.sum(axis = 0)/m
    
    if p > 2:
        # ... = w*normed_gradu**(p-2)
        a1 = W * (normed_gradu**(p-2)) + np.zeros((k, n, n)) # a1[s,i,r] = ...[i, r]
        A = np.transpose(a1, (1, 2, 0)) # A[i,r, s] = ...[i, r]

        u1 = np.zeros( (n, n, k) ) + u #u1[i, r, s] = u[r, s]

        u21 = np.zeros( (n,n,k) ) + u # u21[r, i, s] = u[i, s]
        u2 = np.swapaxes(u21, 0, 1) # u2[i, r, s] = u[i, s]

        C = u1 - u2

        B = np.zeros( (n, k) )
        B[idx] = y - y_bar

        jac = (A * C).sum(axis = 0) - B
    
    elif p == 2:
        a1 = W + np.zeros((k, n, n)) # a1[s,i,r] = W[i, r]
        A = np.transpose(a1, (1, 2, 0)) # A[i, r, s] = W[i, r]
        
        u1 = np.zeros( (n, n, k) ) + u #u1[i, r, s] = u[r, s]

        u21 = np.zeros( (n,n,k) ) + u # u2[r, i, s] = u[i, s]
        u2 = np.swapaxes(u21, 0, 1)

        C = u1 - u2

        B = np.zeros( (n, k) )
        B[idx] = y - y_bar
            
        jac = (A * C).sum(axis = 0) - B
    
    return jac.flatten()

def penergy(u_flattened, W, idx, y, p):
    k = y.shape[1]
    n = int(u_flattened.size/k)
    u = u_flattened.reshape((n,k))
    gradu = graph_grad(u)
    y_bar = (1/y.shape[0]) * y.sum(axis = 0)
    
    first_summand = (1/(2*p)) * (W * (np.apply_along_axis(np.linalg.norm, 2, gradu) ** p)).sum()
    second_summand = np.sum( (y - y_bar) * u[idx] )

    return first_summand - second_summand

def graph_grad(u):
    gradu = -u[:, np.newaxis] + u
    return gradu

def degrees(W):
    return np.ravel((W.sum(axis = 1)))

def predict(u):
    return np.argmax(u, axis = 1)
    

# Solves the p-poisson equation on a graph with weight matrix W
# and labels y on the elements with indices idx
# using gradient descent
def gradient_ppoisson(W, idx, y, p, start = np.zeros(1)):
    d = degrees(W)
    n = W.shape[0]
    k = y.shape[1]
    eye = np.eye(k)

    labels = predict(y)

    if not start.all():
        model = gl.ssl.poisson(W, solver='gradient_descent')
        start = model.fit(idx, labels).flatten()
    
    constrain_matrix = np.concatenate([d[i] * eye for i in range(n)], axis = 1)
    linear_constraint = LinearConstraint(constrain_matrix, np.zeros(k), np.zeros(k))
    
    res = minimize(penergy, x0 = start, args = (W, idx, y, p), jac = vectorized_jacobian, method = 'trust-constr', constraints = linear_constraint)
    
    u = res.x.reshape(n,k)
    
    return u

class ppoisson():
    def __init__(self, W, p = 2):
        self.W = W
        self.p = p
        self.solution = None
        self.predictions = None
        self.runtime = None
        
    def fit(self, idx, euclidean_labels, start = None):
        if not np.array_equiv(self.solution, None):
            return self.solution

        start_time = time.time()

        d = degrees(self.W)
        n = self.W.shape[0]
        k = euclidean_labels.shape[1]
        eye = np.eye(k)

        labels = np.argmax(euclidean_labels, axis = 1)

        if np.array_equal(start, None):
            model = gl.ssl.poisson(self.W, solver='gradient_descent')
            start = model.fit(idx, labels).flatten()
        
        constrain_matrix = np.concatenate([d[i] * eye for i in range(n)], axis = 1)
        linear_constraint = LinearConstraint(constrain_matrix, np.zeros(k), np.zeros(k))
        
        res = minimize(penergy, x0 = start, args = (self.W, idx, euclidean_labels, self.p), jac = vectorized_jacobian, method = 'trust-constr', constraints = linear_constraint)
        
        u = res.x.reshape(n,k)
    
        self.solution = u
        
        end_time = time.time()
        self.runtime = (end_time - start_time)/60

        return self.solution

    def predict(self):
        if np.array_equal(self.solution, None):
            raise ValueError("The model has not been fit yet")
        return np.argmax(self.solution, axis = 1)

    def fit_predict(self, idx, euclidean_labels, start = None):
        if np.array_equal(self.solution, None):
            self.fit(idx, euclidean_labels)
            return self.predict()
        
        return self.predict()



In [3]:
sample_size = 100
X,labels = gl.datasets.load('mnist')
X = X[:sample_size]
labels = labels[:sample_size]

W = gl.weightmatrix.knn(X,5).toarray()
train_ind = gl.trainsets.generate(labels, rate=3, seed = 1)
train_labels = labels[train_ind]
euclidean_labels = euclidean_basis(train_labels, 10)

In [4]:
model = gl.ssl.poisson(W, solver='gradient_descent')
u = model.fit(train_ind, train_labels)
pred_labels = model.predict()

pp_model = ppoisson(W, 2)
my_pred_labels = pp_model.fit_predict(train_ind, euclidean_labels)

In [5]:
print("Energy of GraphLearning Solution: %.3f"%penergy(u.flatten(), W, train_ind, euclidean_labels, 2))
print("Energy of custom solution: %.3f"%penergy(pp_model.solution.flatten(), W, train_ind, euclidean_labels, 2))
print("Accuracy of custom solution: %.2f%%"%gl.ssl.ssl_accuracy(pp_model.predict(), labels, len(train_ind)))
print("Accuracy of GraphLearn solution: %.2f%%"%gl.ssl.ssl_accuracy(pred_labels, labels, len(train_ind)))
print("Number of different predictions: ", np.count_nonzero(pp_model.predict() - pred_labels))
print("Runtime of custom solution: %.2fm"%pp_model.runtime)

Energy of GraphLearning Solution: -517.487
Energy of custom solution: -552.914
Accuracy of custom solution: 71.43%
Accuracy of GraphLearn solution: 71.43%
Number of different predictions:  2
Runtime of custom solution: 0.55m


In [6]:
p_vals = [2, 4, 8, 16, 32, 64, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300]
k = 10

try:
    models
except:
    models = {}
else:
    pass

models['GraphLearning'] = gl.ssl.poisson(W, solver='gradient_descent')
u = models['GraphLearning'].fit(train_ind, train_labels)
pred_labels = predict(u)
my_u = u

# Run and plot for varying p
for p in p_vals:
    models[p] = ppoisson(W, p)
    my_u = models[p].fit(train_ind, euclidean_labels, my_u.flatten())
    my_pred_labels = models[p].predict()

    discrepancies = np.count_nonzero(my_pred_labels - pred_labels)
    accuracy = gl.ssl.ssl_accuracy(my_pred_labels, labels, len(train_ind))
    energy = np.around(penergy(models[p].solution.flatten(), W, train_ind, euclidean_labels, p), 2)
    
    info_str = f"########### Gradient Descent (w/ Jacobian) for p = {p}\n"\
                    f"Energy = {energy:.2f}\n"\
                    f"Discrepancies = {discrepancies}"\
                    f"\nAccuracy = {accuracy:.2f}%\n"\
                    f"Runtime = {models[p].runtime:.2f} min"

    print(info_str)
    

########### Gradient Descent (w/ Jacobian) for p = 2
Energy = -552.91
Discrepancies = 2
Accuracy = 71.43%
Runtime = 0.46 min
########### Gradient Descent (w/ Jacobian) for p = 4
Energy = -100.56
Discrepancies = 7
Accuracy = 67.14%
Runtime = 1.29 min
########### Gradient Descent (w/ Jacobian) for p = 8
Energy = -64.26
Discrepancies = 10
Accuracy = 67.14%
Runtime = 3.03 min
########### Gradient Descent (w/ Jacobian) for p = 16
Energy = -53.50
Discrepancies = 13
Accuracy = 64.29%
Runtime = 3.02 min
########### Gradient Descent (w/ Jacobian) for p = 32
Energy = -49.11
Discrepancies = 12
Accuracy = 65.71%
Runtime = 3.04 min
########### Gradient Descent (w/ Jacobian) for p = 64
Energy = -18.49
Discrepancies = 12
Accuracy = 65.71%
Runtime = 2.91 min
########### Gradient Descent (w/ Jacobian) for p = 100
Energy = 2289.61
Discrepancies = 12
Accuracy = 65.71%
Runtime = 0.02 min
########### Gradient Descent (w/ Jacobian) for p = 110
Energy = 8395.03
Discrepancies = 12
Accuracy = 65.71%
Runtime = 

In [15]:
print(np.linalg.norm(models[300].solution - models[290].solution))
print(np.linalg.norm(models[300].solution - models[150].solution))

0.0
0.0


#Load models

In [7]:
#load_models()

#Save models

In [12]:
#save_models(models)